In [14]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

# Loop

语法：
```
<<lable>>
loop
    statements;
end loop;
```

可以在 `loop` 语句中使用 `exit` 语句来退出循环，也可以使用 `if` 等条件语句来控制循环的执行。

```
loop
    statements;
    if condition then
        exit;
    end if;
end loop;
```

`loop` 也可嵌套
```
<<outer>>
loop
    statements;
    <<inner>>
    loop
        statements;
        if condition then
            exit <<inner>>;
        end if;
    end loop;
end loop;

In [2]:
# 基本用法
sql = """
do $$
declare
    counter int := 0;
begin
  loop
  	counter = counter + 1;
	raise notice '%', counter;
	
	if counter = 5 then
		exit;
	end if;
  end loop;
end;
$$;
"""
cursor.execute(sql)

1
2
3
4
5


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26212c3b110>

In [3]:
# 在实际中，可以把 `if` 与 `exit` 结合使用
sql = """
do $$
declare
    counter int := 0;
begin
  loop
  	counter = counter + 1;
	raise notice '%', counter;
	exit when counter = 5;
  end loop;
end;
$$;
"""
cursor.execute(sql)

1
2
3
4
5


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26212c3b110>

In [4]:
# 使用 label
sql = """
do $$
declare
    counter int := 0;
begin
 <<my_loop>>
  loop
  	counter = counter + 1;
	raise notice '%', counter;
	exit my_loop when counter = 5;
  end loop;
end;
$$;
"""
cursor.execute(sql)

1
2
3
4
5


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26212c3b110>

In [5]:
# 嵌套 loop
sql = """
do $$
declare
	row_var int := 0;
	col_var int := 0;
begin
	<<outer_loop>>
	loop
		row_var = row_var + 1;
		<<inner_loop>>
		loop
			col_var = col_var + 1;
			raise notice '(%, %)', row_var, col_var;
			
			-- terminate the inner loop
			exit inner_loop when col_var = 3;
		end loop;
		-- reset the column
		col_var = 0;
		
		-- terminate the outer loop
		exit outer_loop when row_var = 3;
	end loop;
end;
$$;
"""
cursor.execute(sql)


(1, 1)
(1, 2)
(1, 3)
(2, 1)
(2, 2)
(2, 3)
(3, 1)
(3, 2)
(3, 3)


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26212c3b110>

# While Loop

基本语法：
```
[ <<label>> ]
while condition loop
    statements;
end loop;
```

在进入 `loop` 前，计算 `condition` 的值，如果为 `true`，则执行 `statements`，然后再次计算 `condition` 的值，直到 `condition` 为 `false` 为止。
在 `while` 语句中，我们需要改变 `condition` 的值，否则会陷入死循环。

In [6]:
# 基本用法
sql = """
do $$ 
declare 
	counter integer := 0;
begin 
	while counter < 5 loop 
		raise notice 'Counter %', counter;
		counter := counter + 1;
	end loop;
end;
$$;
"""
cursor.execute(sql)

Counter 0
Counter 1
Counter 2
Counter 3
Counter 4


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26212c3b110>

# For Loop

基本语法：
```
[ <<label>> ]
for loop_counter in [ reverse ] from.. to [ by step ] loop
    statements
end loop [ label ];
```

In [7]:
# 基本用法
sql = """
do 
$$
begin
   for counter in 1..5 loop
	raise notice 'counter: %', counter;
   end loop;
end; 
$$;
"""
cursor.execute(sql)

counter: 1
counter: 2
counter: 3
counter: 4
counter: 5


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26212c3b110>

In [15]:
# 倒序
sql = """
do 
$$
begin
   for counter in reverse 5..1 loop
	raise notice 'counter: %', counter;
   end loop;
end; 
$$;
"""
cursor.execute(sql)

counter: 5
counter: 4
counter: 3
counter: 2
counter: 1


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26214270300>

In [16]:
# 指定步长
sql = """
do $$
begin 
  for counter in 1..6 by 2 loop
    raise notice 'counter: %', counter;
  end loop;
end; $$
"""
cursor.execute(sql)

counter: 1
counter: 3
counter: 5


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26214270300>

## For Loop with Record

基本语法：
```
[ <<label>> ]
for target in query loop
    statements
end loop [ label ];
```

In [17]:
# 基本用法
sql = """
do
$$
declare
    f record;
begin
    for f in select title, length 
	       from film 
	       order by length desc, title
	       limit 10 
    loop 
	raise notice '%(% mins)', f.title, f.length;
    end loop;
end;
$$
"""
cursor.execute(sql)

Chicago North(185 mins)
Control Anthem(185 mins)
Darn Forrester(185 mins)
Gangs Pride(185 mins)
Home Pity(185 mins)
Muscle Bright(185 mins)
Pond Seattle(185 mins)
Soldiers Evolution(185 mins)
Sweet Brotherhood(185 mins)
Worst Banger(185 mins)


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26214270300>

## For loop to iterate over the result set of a dynamic query

基本语法：
```
[ <<label>> ]
for row in execute query_expression [ using query_param [, ... ] ] 
loop
    statements
end loop [ label ];
```

In [18]:
# 基本用法
sql = """
do $$
declare
    -- sort by 1: title, 2: release year
    sort_type smallint := 1; 
	-- return the number of films
	rec_count int := 10;
	-- use to iterate over the film
	rec record;
	-- dynamic query
    query text;
begin
	query := 'select title, release_year from film ';
	if sort_type = 1 then
		query := query || 'order by title';
	elsif sort_type = 2 then
	  query := query || 'order by release_year';
	else 
	   raise 'invalid sort type %s', sort_type;
	end if;

	query := query || ' limit $1';
	for rec in execute query using rec_count
        loop
	     raise notice '% - %', rec.release_year, rec.title;
	end loop;
end;
$$
"""
cursor.execute(sql)

2006 - Academy Dinosaur
2006 - Ace Goldfinger
2006 - Adaptation Holes
2006 - Affair Prejudice
2006 - African Egg
2006 - Agent Truman
2006 - Airplane Sierra
2006 - Airport Pollock
2006 - Alabama Devil
2006 - Aladdin Calendar


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x26214270300>